In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_biweekly_hist
(
  calday DATE,
  store_id STRING,
  product_id STRING,
  replenishment_mode STRING,
  assortment_type STRING,
  pog_mdq_approval_status STRING,
  mdq_source STRING,
  mdq FLOAT,
  mdq_age BIGINT,
  mdq_available STRING,
  external_report_mdq FLOAT,
  internal_report_mdq FLOAT,
  old_mdq FLOAT
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
 """
)

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_biweekly_hist WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS);
""")

In [0]:
spark.sql(f"""INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_biweekly_hist
SELECT 
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) calday, 
    store_id,
    product_id,
    replenishment_mode,
    assortment_type,
    pog_mdq_approval_status,
    mdq_source,
    mdq,
    mdq_age,
    mdq_available,
    external_report_mdq,
    internal_report_mdq,
    old_mdq
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_biweekly;""")